In [1]:
import math
import numpy as np
import pandas as pd
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Tree definition (reused from baseline)
def create_tree_polygon(x, y, angle_deg):
    """Create tree polygon at given position and angle."""
    # Tree vertices (relative to center at origin)
    trunk_w = 0.15
    trunk_h = 0.2
    base_w = 0.7
    mid_w = 0.4
    top_w = 0.25
    tip_y = 0.8
    tier_1_y = 0.5
    tier_2_y = 0.25
    base_y = 0.0
    trunk_bottom_y = -trunk_h
    
    vertices = [
        (0.0, tip_y),
        (top_w/2, tier_1_y),
        (top_w/4, tier_1_y),
        (mid_w/2, tier_2_y),
        (mid_w/4, tier_2_y),
        (base_w/2, base_y),
        (trunk_w/2, base_y),
        (trunk_w/2, trunk_bottom_y),
        (-trunk_w/2, trunk_bottom_y),
        (-trunk_w/2, base_y),
        (-base_w/2, base_y),
        (-mid_w/4, tier_2_y),
        (-mid_w/2, tier_2_y),
        (-top_w/4, tier_1_y),
        (-top_w/2, tier_1_y),
    ]
    
    # Create polygon
    poly = Polygon(vertices)
    
    # Rotate
    poly = affinity.rotate(poly, angle_deg, origin=(0, 0))
    
    # Translate
    poly = affinity.translate(poly, x, y)
    
    return poly

def validate_packing(trees):
    """
    Validate that no trees overlap.
    trees: list of shapely Polygons
    Returns: valid (bool), overlapping_indices (list of tuples)
    """
    valid = True
    overlaps = []
    n = len(trees)
    
    # Naive O(N^2) check - sufficient for N=200
    for i in range(n):
        for j in range(i + 1, n):
            if trees[i].intersects(trees[j]):
                # Check if it's just touching (intersection is line or point)
                intersection = trees[i].intersection(trees[j])
                if intersection.area > 1e-9: # Tolerance for floating point
                    valid = False
                    overlaps.append((i, j))
                    
    return valid, overlaps

def calculate_exact_score(trees, n):
    """
    Calculate score using the bounding box of the union of polygons.
    """
    if not trees:
        return 0
    
    # Union of all trees
    # unary_union is efficient
    combined = unary_union(trees)
    
    min_x, min_y, max_x, max_y = combined.bounds
    
    side = max(max_x - min_x, max_y - min_y)
    score = side**2 / n
    return score

print("Validation and scoring functions defined.")

Validation and scoring functions defined.
